In [1]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

In [3]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa69.deriv.dev/websockets/v3?app_id=16929&l=EN&brand=deriv"
#api token
api_token = "yO3kHGwRauw84xX"

In [5]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    
    if duration_unit == "d":
        
        barrier = "1000"
        
    else:
        
        barrier = "+0"
        
    return_json = []

    def on_open(ws):
        
        msg = {
            "proposal": 1,
            "amount": 60,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        
        amount = 60
    
        message = json.loads(message)
        
        print(message)
        
        if 'error' in message.keys():
            
            barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                
            if message['error']['details']['field'] == 'amount':
                
                amount = float(message['error']['details']['max_stake'])
                print('amount =',amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sent')
            ws.send(json.dumps(msg))   
           
        else:

            return_json.append(message)
            ws.close()
            
    def on_close(ws, close_status_code, close_msg):
        print()             
 
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    barriers = return_json[-1]['proposal']['barrier_choices']

    return return_json[-1]

In [7]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):
    
    spot = float(pd_series.Spot)
    
    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    digits = int( assets[assets['Asset']== symbol].digits )
    
    sigma = float( assets[assets['Asset']==symbol].IV )  
    
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1
    
    spot = round(spot/10**-n_digits)*10**-n_digits
    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400       
    
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if duration_unit != 'd':
        
        deltas = np.arange(1,10,2)/10 
    
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )
    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )
        
    else:
        
        temp_barriers = []
        
        displayed_barriers = []

        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )

        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10

        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 

        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)

        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)

        temp_barriers.append(max_strike)

        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)


    return displayed_barriers
        

In [8]:
def calc_ppp_ask (pd_series, t_step, s_step):

    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    stake = pd_series.Stake

    vol_charge = 1.025
 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400  
    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400

    r = pd_series.Discount_rate 
    
    contract_type = pd_series.Contract_type

    number_of_contracts = pd_series.API_PPP
        
    if s_step == 0:
        
        spot = float(pd_series.Spot)

        strike = pd_series.Barrier
    
    else:
        
        spot = float(tick_history(int(duration[0]), symbol, s_step))
        
        if duration_unit != 'd':
            
            strike = spot + float(pd_series.Displayed_barrier)
            
        else:
            
            strike = pd_series.Barrier
        
    sigma = float( assets[assets['Asset']==symbol].IV )
    
    if duration_unit != 'd':
        
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
        
    else:
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
        
    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))
    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))
    
    return stake / ( bs_price + delta_charge + bs_markup )

In [9]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [10]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [92]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    
    barrier = 0
    
    if duration_unit == "d":
        barrier = "1000"
    else:
        barrier = "+0"
        
    return_json = []

    def on_open(ws):
        
        msg = {
            "proposal": 1,
            "amount": 60,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        if duration_unit == "d":
            barrier = "1000"
        else:
            barrier = "+0"        
        print('\n',message)

        amount = 60

        message = json.loads(message)
        
        if 'error' in message.keys():
            
            if message['error']['details']['field'] == 'barrier':
                
                barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                
            elif message['error']['details']['field'] == 'amount':
                amount = int(message['error']['details']['min_stake']) + 5
                barrier = message['echo_req']['barrier']
                if message['error']['details']['min_stake'] > 2000:
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                    amount = 2
            
            print(amount)
            print(barrier)
            print(contract_type)
            print(duration)
            print(duration_unit)
            print(symbol)
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print(msg)
            ws.send(json.dumps(msg))   
           
        else:

            return_json.append(message)
            ws.close()
            
    def on_close(ws, close_status_code, close_msg):
        print()             
 
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    barriers = return_json[-1]['proposal']['barrier_choices']
    
    if 'qa' in master_URL:
        
        return barriers, return_json[-1]['proposal']['pricing_args']['discount_rate']
    
    else:
        
        return barriers, 'prod_api'

In [79]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []

    return_barrier = []
    
    return_discount_rate = []

    contract_ID = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        print('\n',message)
        barrier = 0
        
        amount = 60
        
        message = json.loads(message)
        
#         print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    

        elif 'error' in message.keys():
#             print('here3')
#             print('stuck error',message)
#             barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            
#             barrier = barriers[0][np.random.randint(len(barriers[0]))]
            
#             return_barrier.append(barrier)
            
#             return_discount_rate.append(barriers[1])
#             print('here')
            if 'code' in message['error'].keys():
                print('here')
                print(message['error']['message'])
                print(message['error']['message'].split()[-1][:-1])
                print(amount)
                amount = float(message['error']['message'].split()[-1][:-1]) 
                
                if 'ax' in message['error']['message']:
                    amount -= 5
                else: 
                    amount += 5
                    
                print('here',amount)
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)

            barrier = barriers[0][np.random.randint(len(barriers[0]))]

            return_barrier.append(barrier)

            return_discount_rate.append(barriers[1])
            
            print (symbol)
            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }

            ws.send(json.dumps(msg))
            
        else:
            return_json.append(message)
            
            ws.close()
                                          
    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [94]:
def calc_ppp_bid (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['bid_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400  
        
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if t > 1 / 365:
        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    
    
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['current_spot'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))
        
        if t > 1 / 365:
        
            strike = float(proposal['proposal_open_contract']['barrier'])

        else:

            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

        
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)

    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))

    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts
    

In [152]:
def sell_contract(contract_id):
    
    contract_id = int(contract_id)
    
    temp_json = []
    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]

In [153]:
def calc_ppp_sold (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['sell_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  

    if t > 1 / 365:
        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['exit_tick'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)


    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return price * number_of_contracts
    


In [140]:
def calc_ppp_exp (proposal, t_step, s_step):

    
    stake = float(proposal['proposal_open_contract']['sell_price'])
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['exit_tick'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    if contract_type == "VANILLALONGCALL":
        phi = 1

    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    
    
        
    print(spot,strike, phi,  number_of_contracts, stake)

    return max( (spot - strike) * phi * number_of_contracts, 0)
    


In [19]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    clear_output(wait = True)
    
    print(len(barrier_df))
    if len(barrier_df) > 30 - 1:
        break
        
        
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        
        max_duration = 1440

    elif duration_unit == 'h':
        
        max_duration = 24

    elif duration_unit == 'd':
        
        max_duration = 365

    i = np.random.randint(5)
    
    if i < 2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1,max_duration)
        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)
    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]
    
    time.sleep(0.3)
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,1HZ100V,VANILLALONGCALL,1025.47,1683701639,1440m,"[1683701639, 1683788039]","[73.0, 30.1, 1.4, -26.5, -65.5]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+1.40', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ100V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrie..."
1,R_100,VANILLALONGCALL,5195.25,1683701640,1m,"[1683701642, 1683701702]","[9.1, 3.7, 0.0, -3.7, -9.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_cho..."
2,1HZ75V,VANILLALONGCALL,7060.87,1683701644,1h,"[1683701644, 1683705244]","[73.5, 30.1, 0.2, -29.5, -72.3]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.20', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ75V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_ch..."
3,R_75,VANILLALONGPUT,376221.9664,1683701646,56d,"[1683701647, 1688601599]","[280000.0, 300000.0, 320000.0, 340000.0, 360000.0, 380000.0, 418000.0, 456000.0, 494000.0, 532000.0, 570000.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '380000.0000', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 56, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_75'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrie..."
4,1HZ75V,VANILLALONGCALL,7059.25,1683701649,674m,"[1683701650, 1683742090]","[251.3, 103.3, 2.6, -96.8, -237.7]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+2.60', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 674, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ75V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_..."
5,R_100,VANILLALONGPUT,5196.70,1683701652,1m,"[1683701652, 1683701712]","[9.1, 3.7, 0.0, -3.7, -9.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_choi..."
6,1HZ25V,VANILLALONGCALL,453738.26,1683701655,24h,"[1683701655, 1683788055]","[7717.2, 3165.2, 38.9, -3066.1, -7511.8]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+38.90', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 24, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_..."
7,R_75,VANILLALONGCALL,376463.0396,1683701656,20h,"[1683701657, 1683773657]","[18114.46, 7457.434, 244.085, -6834.972, -16824.133]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+244.0850', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 20, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_75'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier..."
8,R_10,VANILLALONGCALL,6671.629,1683701660,362d,"[1683701660, 1715039999]","[5900.0, 6050.0, 6200.0, 6350.0, 6500.0, 6670.0, 6860.0, 7050.0, 7240.0, 7430.0, 7610.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '6670.000', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 362, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_10'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier..."
9,1HZ75V,VANILLA

In [50]:
Calc_barrier_choices = [] 

for i in barrier_df.index:
    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )

barrier_df.Calc_barrier_choices = Calc_barrier_choices

print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) != 0]),\
      '\nMismatches listed below :')

barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) != 0]


30 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [168]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    clear_output(wait=True)
    print(len(PPP_df))
    
    if len(PPP_df)>100-1:
        break
        
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_50'
    duration_unit = random.choice(time_units)
#     duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
    
    if i<2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1, max_duration)
#     duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
                               

#     clear_output(wait=True)
clear_output(wait=True)

PPP_df


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,1HZ25V,VANILLALONGPUT,453882.0800,1683702001,60.00,1h,+1.60,453883.6800,"[1683702001, 1683705601]",5139,0.000000,0.120833,0.120833,"{'echo_req': {'contract_id': 5139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '453883.68', 'barrier_count': 1, 'bid_price': 57.08, 'buy_price': '60.00', 'contract_id': '5139', 'contract_type': 'VANILLALONGPUT'..."
1,R_50,VANILLALONGPUT,153.2821,1683702006,60.00,1d,159.0000,159.0000,"[1683702006, 1683849599]",5179,0.050582,9.779930,9.779930,"{'echo_req': {'contract_id': 5179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '159.0000', 'barrier_count': 1, 'bid_price': 59.41, 'buy_price': '60.00', 'contract_id': '5179', 'contract_type': 'VANILLALONGPUT',..."
2,R_50,VANILLALONGCALL,153.2653,1683702010,60.00,8h,-2.9170,150.3483,"[1683702011, 1683730811]",5219,0.000000,19.748186,19.748186,"{'echo_req': {'contract_id': 5219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '150.3483', 'barrier_count': 1, 'bid_price': '59.60', 'buy_price': '60.00', 'contract_id': '5219', 'contract_type': 'VANILLALONGCAL..."
3,1HZ50V,VANILLALONGCALL,776424.7400,1683702019,206.13,818m,+8263.40,784688.1400,"[1683702019, 1683751099]",5279,0.000000,0.068529,0.068529,"{'echo_req': {'contract_id': 5279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '784688.14', 'barrier_count': 1, 'bid_price': 185.53, 'buy_price': 206.13, 'contract_id': '5279', 'contract_type': 'VANILLALONGCALL..."
4,1HZ50V,VANILLALONGPUT,776236.3600,1683702024,60.00,1m,+0.20,776236.5600,"[1683702025, 1683702085]",5319,0.000000,0.273957,0.273957,"{'echo_req': {'contract_id': 5319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '776236.56', 'barrier_count': 1, 'bid_price': 65.47, 'buy_price': '60.00', 'contract_id': '5319', 'contract_type': 'VANILLALONGPUT'..."
5,1HZ25V,VANILLALONGCALL,453877.1900,1683702032,83.21,1440m,+3165.20,457042.3900,"[1683702032, 1683788432]",5359,0.000000,0.070847,0.070847,"{'echo_req': {'contract_id': 5359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '457042.39', 'barrier_count': 1, 'bid_price': 76.04, 'buy_price': 83.21, 'contract_id': '5359', 'contract_type': 'VANILLALONGCALL',..."
6,R_10,VANILLALONGCALL,6671.3760,1683702036,60.00,365d,7620.000,7620.0000,"[1683702037, 1715299199]",5399,0.050162,1.879351,1.879351,"{'echo_req': {'contract_id': 5399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '7620.000', 'barrier_count': 1, 'bid_price': 49.48, 'buy_price': '60.00', 'contract_id': '5399', 'contract_type': 'VANILLALONGCALL'..."
7,1HZ25V,VANILLALONGPUT,453879.6900,1683702042,60.00,7h,+4141.90,458021.5900,"[1683702042, 1683727242]",5439,0.000000,0.013929,0.013929,"{'echo_req': {'contract_id': 5439, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '458021.59', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': '60.00', 'contract_id': '5439', 'contract_type': 'VANILLALONGPUT'..."
8,R_50,VANILLALONGCALL,153.1710,1683702046,60.00,624m,+1.4110,154.5820,"[1683702047, 1683739487]",5479,0.000000,115.060714,115.060714,"{'echo_req': {'contract_id': 5479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '154.5820', 'barrier_count': 1, 'bid_price': 54.72, 'buy_price': '60.00', 'contrac

In [51]:
Calc_ppp = [] 

for i in PPP_df.index:
    
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )

PPP_df.Calc_PPP = Calc_ppp

clear_output(wait = True)

print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]),\
      '\nMismatches listed below :')

df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]

df

21 Test cases 
Mismatches = 1 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
18,R_10,VANILLALONGCALL,6674.043,1683702186,60.0,91d,6850.000,6850.0,"[1683702187, 1691625599]",6019,0.053396,0.894178,0.894178,"{'echo_req': {'contract_id': 6019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200121', 'barrier': '6850.000', 'barrier_count': 1, 'bid_price': 54.79, 'buy_price': '60.00', 'contract_id': '6019', 'contract_type': 'VANILLALONGCALL'..."


In [169]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    
    print(len(Bid_df))
    
    if len(Bid_df)>100-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_10'
    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               

    clear_output(wait=True)
Bid_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ25V,VANILLALONGPUT,446653.9500,1683772977,60.00,246d,351000.00,351000.0000,"[1683772977, 1705103999]",7899,0.051610,0.011878,1683775642,51.60,51.599935,"{'echo_req': {'contract_id': 7899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '351000.00', 'barrier_count': 1, 'bid_price': '51.60', 'buy_price': '60.00', 'contract_id': '7899', 'contract_type': 'VANILLALONGPU...","{'echo_req': {'contract_id': 7899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '351000.00', 'barrier_count': 1, 'bid_price': 50.68, 'buy_price': '60.00', 'contract_id': '7899', 'contract_type': 'VANILLALONGPUT'..."
1,1HZ100V,VANILLALONGPUT,998.4200,1683773019,60.00,15h,+0.90,999.3200,"[1683773020, 1683827020]",7919,0.000000,3.457320,1683775644,65.65,65.654666,"{'echo_req': {'contract_id': 7919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '999.32', 'barrier_count': 1, 'bid_price': 65.65, 'buy_price': '60.00', 'contract_id': '7919', 'contract_type': 'VANILLALONGPUT', '...","{'echo_req': {'contract_id': 7919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '999.32', 'barrier_count': 1, 'bid_price': 56.79, 'buy_price': '60.00', 'contract_id': '7919', 'contract_type': 'VANILLALONGPUT', '..."
2,1HZ75V,VANILLALONGPUT,7579.5700,1683773024,60.00,723m,+114.70,7694.2700,"[1683773025, 1683816405]",7939,0.000000,0.384630,1683775647,55.76,55.756918,"{'echo_req': {'contract_id': 7939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7694.27', 'barrier_count': 1, 'bid_price': 55.76, 'buy_price': '60.00', 'contract_id': '7939', 'contract_type': 'VANILLALONGPUT', ...","{'echo_req': {'contract_id': 7939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7694.27', 'barrier_count': 1, 'bid_price': 58.69, 'buy_price': '60.00', 'contract_id': '7939', 'contract_type': 'VANILLALONGPUT', ..."
3,R_100,VANILLALONGPUT,4904.2900,1683773030,60.00,1404m,-307.90,4596.3900,"[1683773030, 1683857270]",7959,0.000000,4.467940,1683775648,56.60,56.603146,"{'echo_req': {'contract_id': 7959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '4596.39', 'barrier_count': 1, 'bid_price': '56.60', 'buy_price': '60.00', 'contract_id': '7959', 'contract_type': 'VANILLALONGPUT'...","{'echo_req': {'contract_id': 7959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '4596.39', 'barrier_count': 1, 'bid_price': 50.05, 'buy_price': '60.00', 'contract_id': '7959', 'contract_type': 'VANILLALONGPUT', ..."
4,1HZ100V,VANILLALONGCALL,998.7700,1683773035,60.00,1081m,+60.90,1059.6700,"[1683773035, 1683837895]",7979,0.000000,26.048247,1683775652,36.52,36.518945,"{'echo_req': {'contract_id': 7979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '1059.67', 'barrier_count': 1, 'bid_price': 36.52, 'buy_price': '60.00', 'contract_id': '7979', 'contract_type': 'VANILLALONGCALL',...","{'echo_req': {'contract_id': 7979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '1059.67', 'barrier_count': 1, 'bid_price': 49.37, 'buy_price': '60.00', 'contract_id': '7979', 'contract_type': 'VANILLALONGCALL',..."
5,1HZ100V,VANILLALONGCALL,998.9100,1683773041,60.00,1078m,-22.50,9

In [137]:
# calc_bid = [] 

# bid_time = []

# api_bid = []

# bid_prop = []

for i in Bid_df.index:
    
    if i >= len(bid_time):
        
        clear_output(wait = True)

        proposal = get_proposal(Bid_df.loc[i].ID)

        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])

        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))

        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )
        
        bid_prop.append(proposal)
    
Bid_df.Bid_time = bid_time

Bid_df.API_bid = api_bid

Bid_df.Calc_bid = calc_bid

Bid_df.bid_prop = bid_prop

clear_output(wait = True)

print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[round(Bid_df.API_bid,8) - np.round(Bid_df.Calc_bid,2) != 0]),\
      '\nMismatches listed below :')

df = Bid_df[round(Bid_df.API_bid,8) - np.round(Bid_df.Calc_bid,2) != 0]

df

30 Test cases 
Mismatches = 3 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
11,1HZ50V,VANILLALONGPUT,771790.9800,1683773073,60.00,1160m,-9216.70,762574.2800,"[1683773073, 1683842673]",8139,0.000000,0.016397,1683775669,56.88,56.885234,"{'echo_req': {'contract_id': 8139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '762574.28', 'barrier_count': 1, 'bid_price': 56.88, 'buy_price': '60.00', 'contract_id': '8139', 'contract_type': 'VANILLALONGPUT'...","{'echo_req': {'contract_id': 8139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '762574.28', 'barrier_count': 1, 'bid_price': 54.83, 'buy_price': '60.00', 'contract_id': '8139', 'contract_type': 'VANILLALONGPUT'..."
21,R_50,VANILLALONGCALL,151.1412,1683775530,60.00,14h,-1.5450,149.5962,"[1683775531, 1683825931]",10019,0.000000,27.869068,1683775694,63.53,63.535006,"{'echo_req': {'contract_id': 10019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '149.5962', 'barrier_count': 1, 'bid_price': 63.53, 'buy_price': '60.00', 'contract_id': '10019', 'contract_type': 'VANILLALONGCAL...","{'echo_req': {'contract_id': 10019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '149.5962', 'barrier_count': 1, 'bid_price': 58.54, 'buy_price': '60.00', 'contract_id': '10019', 'contract_type': 'VANILLALONGCAL..."
25,R_75,VANILLALONGPUT,393419.0301,1683775598,1748.65,236d,390000.0000,390000.0000,"[1683775599, 1704239999]",10099,0.051632,0.019329,1683775704,1662.54,1662.545021,"{'echo_req': {'contract_id': 10099, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '390000.0000', 'barrier_count': 1, 'bid_price': 1662.54, 'buy_price': 1748.65, 'contract_id': '10099', 'contract_type': 'VANILLALO...","{'echo_req': {'contract_id': 10099, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '390000.0000', 'barrier_count': 1, 'bid_price': 1664.36, 'buy_price': 1748.65, 'contract_id': '10099', 'contract_type': 'VANILLALO..."


In [96]:
calc_bid = [] 

bid_time = []

api_bid = []

bid_prop = []

for i in df.index:
    
    if i >= len(bid_time):
        
        clear_output(wait = True)

        proposal = df.loc[i].bid_prop

        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])

        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))

        calc_bid.append( calc_ppp_bid(proposal, -1, 0) )
        
        bid_prop.append(proposal)
    
df.Bid_time = bid_time

df.API_bid = api_bid

df.Calc_bid = calc_bid

df.bid_prop = bid_prop

clear_output(wait = True)

print(len(df), 'Test cases','\nMismatches =',\
      len(df[round(df.API_bid,8) - round(df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')

df2 = df[round(df.API_bid,8) - round(df.Calc_bid,8) != 0]

df2

2 Test cases 
Mismatches = 2 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
10,R_25,VANILLALONGPUT,1915.0060,1683773066,60.0,919m,-25.450,1889.5560,"[1683773067, 1683828207]",8119,0.0,58.022352,1683773762,49.19,49.194814,"{'echo_req': {'contract_id': 8119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '1889.556', 'barrier_count': 1, 'bid_price': 49.19, 'buy_price': '60.00', 'contract_id': '8119', 'contract_type': 'VANILLALONGPUT',...","{'echo_req': {'contract_id': 8119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '1889.556', 'barrier_count': 1, 'bid_price': 49.75, 'buy_price': '60.00', 'contract_id': '8119', 'contract_type': 'VANILLALONGPUT',..."
18,R_75,VANILLALONGCALL,391192.9576,1683773176,60.0,13h,+14880.1050,406073.0626,"[1683773176, 1683819976]",8379,0.0,0.102749,1683773780,38.22,38.223358,"{'echo_req': {'contract_id': 8379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '406073.0626', 'barrier_count': 1, 'bid_price': 38.22, 'buy_price': '60.00', 'contract_id': '8379', 'contract_type': 'VANILLALONGCA...","{'echo_req': {'contract_id': 8379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '406073.0626', 'barrier_count': 1, 'bid_price': 49.76, 'buy_price': '60.00', 'contract_id': '8379', 'contract_type': 'VANILLALONGCA..."


In [150]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    
    print(len(Sold_df))
    
    if len(Sold_df) > 20 - 1:
        break
    
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               

    clear_output(wait=True)
    
Sold_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ75V,VANILLALONGCALL,7584.1000,1683789320,60.00,1381m,+5.60,7589.7000,"[1683789320, 1683872180]",10819,0.000000,0.515099,0,0,0,0,"{'echo_req': {'contract_id': 10819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7589.70', 'barrier_count': 1, 'bid_price': 56.85, 'buy_price': '60.00', 'contract_id': '10819', 'contract_type': 'VANILLALONGCALL..."
1,1HZ75V,VANILLALONGCALL,7583.4300,1683789326,60.00,20h,+149.10,7732.5300,"[1683789326, 1683861326]",10839,0.000000,1.131110,0,0,0,0,"{'echo_req': {'contract_id': 10839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7732.53', 'barrier_count': 1, 'bid_price': 54.68, 'buy_price': '60.00', 'contract_id': '10839', 'contract_type': 'VANILLALONGCALL..."
2,1HZ75V,VANILLALONGCALL,7582.7000,1683789331,60.00,1064m,-317.50,7265.2000,"[1683789331, 1683853171]",10859,0.000000,0.181350,0,0,0,0,"{'echo_req': {'contract_id': 10859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7265.20', 'barrier_count': 1, 'bid_price': 59.59, 'buy_price': '60.00', 'contract_id': '10859', 'contract_type': 'VANILLALONGCALL..."
3,1HZ10V,VANILLALONGPUT,9120.8800,1683789336,60.00,13h,-44.80,9076.0800,"[1683789336, 1683836136]",10879,0.000000,32.808299,0,0,0,0,"{'echo_req': {'contract_id': 10879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '9076.08', 'barrier_count': 1, 'bid_price': 49.87, 'buy_price': '60.00', 'contract_id': '10879', 'contract_type': 'VANILLALONGPUT'..."
4,1HZ75V,VANILLALONGCALL,7583.1800,1683789341,60.00,1228m,+151.00,7734.1800,"[1683789342, 1683863022]",10899,0.000000,1.118551,0,0,0,0,"{'echo_req': {'contract_id': 10899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7734.18', 'barrier_count': 1, 'bid_price': 54.33, 'buy_price': '60.00', 'contract_id': '10899', 'contract_type': 'VANILLALONGCALL..."
5,1HZ25V,VANILLALONGCALL,453048.9900,1683789347,60.00,1068m,+2713.90,455762.8900,"[1683789347, 1683853427]",10919,0.000000,0.059356,0,0,0,0,"{'echo_req': {'contract_id': 10919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '455762.89', 'barrier_count': 1, 'bid_price': 54.73, 'buy_price': '60.00', 'contract_id': '10919', 'contract_type': 'VANILLALONGCA..."
6,R_75,VANILLALONGPUT,389950.2106,1683789376,60.00,17h,-15964.5040,373985.7066,"[1683789376, 1683850576]",10939,0.000000,0.088777,0,0,0,0,"{'echo_req': {'contract_id': 10939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '373985.7066', 'barrier_count': 1, 'bid_price': 49.97, 'buy_price': '60.00', 'contract_id': '10939', 'contract_type': 'VANILLALONG..."
7,1HZ100V,VANILLALONGPUT,998.7500,1683789381,60.00,209d,810.00,810.0000,"[1683789382, 1701907199]",10959,0.052268,0.328204,0,0,0,0,"{'echo_req': {'contract_id': 10959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '810.00', 'barrier_count': 1, 'bid_price': 56.13, 'buy_price': '60.00', 'contract_id': '10959', 'contract_type': 'VANILLALONGPUT',..."
8,R_50,VANILLALONGPUT,149.2913,1683789498,60.00,794m,+1.5550,150.8463,"[1683789499, 1683837139]",10979,0.000000,28.197690,0,0,0,0,"{'echo_req': {'contract_id': 10979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '150.8463', 'barrier_count': 1, 'bid_price': 58.58, 'bu

In [160]:
calc_sold = [] 

sold_time = []

api_sold = []

sold_prop = []

for i in Sold_df.index:
    
    if i >= len(sold_time):
        
        clear_output(wait = True)
        
#         sell_contract(Sold_df.loc[i].ID)
        
        proposal = Sold_df.loc[i].Sold_prop

        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])

        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))

        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )
        
        sold_prop.append(proposal)
    
Sold_df['Sold_time'] = sold_time

Sold_df['API_sold'] = api_sold

Sold_df['Calc_sold'] = calc_sold

Sold_df['Sold_prop'] = sold_prop

clear_output(wait = True)

print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[Sold_df.API_sold - np.round(Sold_df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')

df = Sold_df[Sold_df.API_sold - np.round(Sold_df.Calc_sold,2) != 0]

df

20 Test cases 
Mismatches = 4 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop
3,1HZ10V,VANILLALONGPUT,9120.8800,1683789336,60.0,13h,-44.80,9076.0800,"[1683789336, 1683836136]",10879,...,32.808299,0,0,0,0,"{'echo_req': {'contract_id': 10879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '9076.08', 'barrier_count': 1, 'bid_price': 49.87, 'buy_price': '60.00', 'contract_id': '10879', 'contract_type': 'VANILLALONGPUT'...",1683789653,61.78,62.038759,"{'echo_req': {'contract_id': 10879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'audit_details': {'contract_start': [{'epoch': 1683789334, 'tick': '9120.74', 'tick_display_value': '9120.74'}, {'epoch': 1683789335, 'tick':..."
5,1HZ25V,VANILLALONGCALL,453048.9900,1683789347,60.0,1068m,+2713.90,455762.8900,"[1683789347, 1683853427]",10919,...,0.059356,0,0,0,0,"{'echo_req': {'contract_id': 10919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '455762.89', 'barrier_count': 1, 'bid_price': 54.73, 'buy_price': '60.00', 'contract_id': '10919', 'contract_type': 'VANILLALONGCA...",1683789663,59.82,60.079639,"{'echo_req': {'contract_id': 10919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'audit_details': {'contract_start': [{'epoch': 1683789345, 'tick': '453037.4', 'tick_display_value': '453037.40'}, {'epoch': 1683789346, 'tic..."
6,R_75,VANILLALONGPUT,389950.2106,1683789376,60.0,17h,-15964.5040,373985.7066,"[1683789376, 1683850576]",10939,...,0.088777,0,0,0,0,"{'echo_req': {'contract_id': 10939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '373985.7066', 'barrier_count': 1, 'bid_price': 49.97, 'buy_price': '60.00', 'contract_id': '10939', 'contract_type': 'VANILLALONG...",1683789668,51.11,51.007301,"{'echo_req': {'contract_id': 10939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'audit_details': {'contract_start': [{'epoch': 1683789372, 'tick': '389992.0095', 'tick_display_value': '389992.0095'}, {'epoch': 1683789374,..."
17,1HZ75V,VANILLALONGCALL,7578.6900,1683789587,60.0,193d,9620.00,9620.0000,"[1683789588, 1700524799]",11159,...,0.059361,0,0,0,0,"{'echo_req': {'contract_id': 11159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '9620.00', 'barrier_count': 1, 'bid_price': 55.31, 'buy_price': '60.00', 'contract_id': '11159', 'contract_type': 'VANILLALONGCALL...",1683789723,54.80,54.887772,"{'echo_req': {'contract_id': 11159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'audit_details': {'contract_start': [{'epoch': 1683789586, 'tick': '7578.34', 'tick_display_value': '7578.34'}, {'epoch': 1683789587, 'flag':..."


In [139]:
Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
                               'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m')

while True:
    
    print(len(Expired_df))
    
    if len(Expired_df)> 30-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(1, 5)
#     duration=1
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]
                               

    clear_output(wait=True)
Expired_df

30


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,R_50,VANILLALONGCALL,150.1898,1683785654,60.0,3m,+0.0000,150.1898,"[1683785655, 1683785835]",10219,0,822.432388,0,0,"{'echo_req': {'contract_id': 10219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '150.1898', 'barrier_count': 1, 'bid_price': 56.75, 'buy_price': '60.00', 'contract_id': '10219', 'contract_type': 'VANILLALONGCAL..."
1,R_75,VANILLALONGPUT,396781.3723,1683785660,60.0,1m,-520.8500,396260.5223,"[1683785660, 1683785720]",10239,0,2.929449,0,0,"{'echo_req': {'contract_id': 10239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '396260.5223', 'barrier_count': 1, 'bid_price': 49.77, 'buy_price': '60.00', 'contract_id': '10239', 'contract_type': 'VANILLALONG..."
2,R_50,VANILLALONGCALL,150.1288,1683785666,60.0,3m,-0.0930,150.0358,"[1683785666, 1683785846]",10259,0,466.401781,0,0,"{'echo_req': {'contract_id': 10259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '150.0358', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': '60.00', 'contract_id': '10259', 'contract_type': 'VANILLALONGCAL..."
3,1HZ50V,VANILLALONGCALL,777342.2900,1683785671,60.0,2m,+975.90,778318.1900,"[1683785671, 1683785791]",10279,0,1.541258,0,0,"{'echo_req': {'contract_id': 10279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '778318.19', 'barrier_count': 1, 'bid_price': 49.01, 'buy_price': '60.00', 'contract_id': '10279', 'contract_type': 'VANILLALONGCA..."
4,R_100,VANILLALONGCALL,4906.1900,1683785676,60.0,3m,+6.10,4912.2900,"[1683785677, 1683785857]",10299,0,25.783882,0,0,"{'echo_req': {'contract_id': 10299, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '4912.29', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': '60.00', 'contract_id': '10299', 'contract_type': 'VANILLALONGCALL..."
5,1HZ75V,VANILLALONGPUT,7589.5100,1683785682,60.0,1m,-4.10,7585.4100,"[1683785682, 1683785742]",10319,0,38.273283,0,0,"{'echo_req': {'contract_id': 10319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '7585.41', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': '60.00', 'contract_id': '10319', 'contract_type': 'VANILLALONGPUT'..."
6,1HZ10V,VANILLALONGPUT,9138.7800,1683785687,60.0,3m,-1.10,9137.6800,"[1683785687, 1683785867]",10339,0,133.783943,0,0,"{'echo_req': {'contract_id': 10339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '9137.68', 'barrier_count': 1, 'bid_price': 54.59, 'buy_price': '60.00', 'contract_id': '10339', 'contract_type': 'VANILLALONGPUT'..."
7,1HZ50V,VANILLALONGCALL,777271.6600,1683785692,60.0,2m,-398.50,776873.1600,"[1683785692, 1683785812]",10359,0,0.109274,0,0,"{'echo_req': {'contract_id': 10359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '776873.16', 'barrier_count': 1, 'bid_price': 58.45, 'buy_price': '60.00', 'contract_id': '10359', 'contract_type': 'VANILLALONGCA..."
8,R_100,VANILLALONGCALL,4901.3200,1683785696,60.0,3m,-14.90,4886.4200,"[1683785698, 1683785878]",10379,0,3.870337,0,0,"{'echo_req': {'contract_id': 10379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200221', 'barrier': '4886.42', 'barrier_count': 1, 'bid_price': 64.28, 'buy_price': '60.00', 'contract_id': '10379', 'contract_type': 'VANILLALONGCALL..."
9,R_25,VANILLALONGPUT,1913.2670,1683785702,60.0,4m,-1.680,1911.58

In [142]:
calc_exp = [] 

exp_time = []

api_exp = []

for i in Expired_df.index:
    
    if i >= len(exp_time):
        
        clear_output(wait = True)
                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)
        
        if proposal['proposal_open_contract']['is_sold'] == 1:
            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])

            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))

            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )
            
        else:
            
            exp_time.append(0)
            
            api_exp.append(0)

            calc_exp.append(0)
    
Expired_df['Exp_time'] = exp_time

Expired_df['API_exp'] = api_exp

Expired_df['Calc_exp'] = calc_exp

clear_output(wait = True)

print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[round(Expired_df.API_exp,8) - round(Expired_df.Calc_exp,2) != 0]),\
      '\nMismatches listed below :')

df = Expired_df[round(Expired_df.API_exp,8) - round(Expired_df.Calc_exp,2) != 0]

df

30 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
